In [45]:
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import StandardScaler 

In [46]:
dataset = pd.read_csv("/content/forestfires.csv") 

In [47]:
X = dataset.iloc[:,2:30]
Y = dataset.iloc[:,30]

In [48]:
a = StandardScaler()
a.fit(X)
X_Standardized = a.transform(X) 

In [49]:
pd.DataFrame(X_Standardized).describe() 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
count,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02
mean,-1.754024e-15,3.070830e-16,7.387171e-17,-3.865380e-17,2.005703e-16,3.362881e-16,-2.676776e-16,-2.841054e-16,-1.274502e-16,4.874674e-17,-1.868267e-16,-2.238699e-16,-6.098711e-17,-1.004999e-16,2.405125e-17,-3.843906e-17,-1.344293e-16,2.473843e-16,7.179943e-16,-1.933764e-16,-2.260174e-17,1.352883e-17,1.169277e-16,2.265542e-16,-2.596515e-16,1.443075e-16,6.253326e-16,4.024290e-16
std,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00
min,-1.304582e+01,-1.715608e+00,-2.179108e+00,-1.980578e+00,-2.876943e+00,-1.796637e+00,-2.021098e+00,-7.326831e-02,-2.020198e-01,-4.435755e-01,-4.087088e-01,-4.404491e-01,-4.744665e-01,-3.657485e-01,-3.758728e-01,-3.415123e-01,-1.331035e-01,-7.433388e-01,-1.331035e-01,-2.006027e-01,-6.231770e-02,-2.568645e-01,-1.843909e-01,-3.415123e-01,-6.231770e-02,-4.402255e-02,-1.728597e-01,-7.060812e-01
25%,-8.063453e-02,-6.606652e-01,-4.448281e-01,-5.535954e-01,-5.842379e-01,-6.924563e-01,-7.361236e-01,-7.326831e-02,-2.020198e-01,-4.435755e-01,-4.087088e-01,-4.404491e-01,-4.744665e-01,-3.657485e-01,-3.758728e-01,-3.415123e-01,-1.331035e-01,-7.433388e-01,-1.331035e-01,-2.006027e-01,-6.231770e-02,-2.568645e-01,-1.843909e-01,-3.415123e-01,-6.231770e-02,-4.402255e-02,-1.728597e-01,-7.060812e-01
50%,1.732292e-01,-4.020255e-02,4.691190e-01,-1.364774e-01,7.082076e-02,-1.403660e-01,-9.833712e-03,-7.326831e-02,-1.938429e-01,-4.435755e-01,-4.087088e-01,-4.404491e-01,-4.744665e-01,-3.657485e-01,-3.758728e-01,-3.415123e-01,-1.331035e-01,-7.433388e-01,-1.331035e-01,-2.006027e-01,-6.231770e-02,-2.568645e-01,-1.843909e-01,-3.415123e-01,-6.231770e-02,-4.402255e-02,-1.728597e-01,-7.060812e-01
75%,4.089598e-01,4.927389e-01,6.696628e-01,3.904086e-01,6.741643e-01,5.344111e-01,4.929823e-01,-7.326831e-02,-9.870852e-02,-4.435755e-01,-4.087088e-01,-4.404491e-01,-4.744665e-01,-3.657485e-01,-3.758728e-01,-3.415123e-01,-1.331035e-01,1.345282e+00,-1.331035e-01,-2.006027e-01,-6.231770e-02,-2.568645e-01,-1.843909e-01,-3.415123e-01,-6.231770e-02,-4.402255e-02,-1.728597e-01,1.416268e+00
max,1.007353e+00,2.819865e+00,1.261610e+00,1.033538e+01,2.484195e+00,3.417549e+00,3.007063e+00,2.157228e+01,1.695111e+01,2.254407e+00,2.446730e+00,2.270410e+00,2.107630e+00,2.734120e+00,2.660475e+00,2.928152e+00,7.512952e+00,1.345282e+00,7.512952e+00,4.984977e+00,1.604681e+01,3.893103e+00,5.423261e+00,2.928152e+00,1.604681e+01,2.271563e+01,5.785038e+00,1.416268e+00


#### Tuning of Hyperparameters :- Batch Size and Epochs


In [50]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam 

In [61]:
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=28, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    
    adam=Adam(lr=0.01)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [62]:
model = KerasClassifier(build_fn = create_model,verbose = 0)
batch_size = [10,20,40]
epochs = [10,50,100]
param_grid = dict(batch_size = batch_size,epochs = epochs)
grid = GridSearchCV(estimator = model,param_grid = param_grid,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_Standardized,Y) 

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............ batch_size=10, epochs=10, score=0.990, total=   1.5s
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s


[CV] ............ batch_size=10, epochs=10, score=0.827, total=   1.0s
[CV] batch_size=10, epochs=10 ........................................


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   13.4s remaining:    0.0s


[CV] ............ batch_size=10, epochs=50, score=0.903, total=   2.7s
[CV] batch_size=10, epochs=50 ........................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   16.0s remaining:    0.0s


[CV] ............ batch_size=10, epochs=50, score=0.893, total=   2.8s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.990, total=   4.2s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.856, total=   4.2s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.893, total=   4.3s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.893, total=   4.2s
[CV] batch_size=10, epochs=100 .......................................
[CV] ........... batch_size=10, epochs=100, score=0.893, total=   4.2s
[CV] batch_size=20, epochs=10 ........................................
[CV] ............ batch_size=20, epochs=10, score=0.990, total=   0.8s
[CV] batch_size=20, epochs=10 ........................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.4min finished


In [63]:
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param)) 

Best : 0.9167475700378418, using {'batch_size': 20, 'epochs': 50}
0.8721994042396546,0.06252856759191373 with: {'batch_size': 10, 'epochs': 10}
0.9070761680603028,0.04905727739713657 with: {'batch_size': 10, 'epochs': 50}
0.9051530838012696,0.04501452303654429 with: {'batch_size': 10, 'epochs': 100}
0.870238983631134,0.06385404247649518 with: {'batch_size': 20, 'epochs': 10}
0.9167475700378418,0.043443214944303685 with: {'batch_size': 20, 'epochs': 50}
0.9070948481559753,0.05123596344455126 with: {'batch_size': 20, 'epochs': 100}
0.7791635513305664,0.14826411278207174 with: {'batch_size': 40, 'epochs': 10}
0.8954630255699157,0.05573770684810709 with: {'batch_size': 40, 'epochs': 50}
0.914805817604065,0.04631027544291211 with: {'batch_size': 40, 'epochs': 100}


Tuning of Hyperparameters:- Learning rate and Drop out rate

In [64]:
from keras.layers import Dropout

# Defining the model

def create_model(learning_rate,dropout_rate):
    model = Sequential()
    model.add(Dense(8,input_dim = 28,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4,input_dim = 28,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = learning_rate)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 10)

# Define the grid search parameters

learning_rate = [0.001,0.01,0.1]
dropout_rate = [0.1,0.2,0.3]

# Make a dictionary of the grid search parameters

param_grids = dict(learning_rate = learning_rate,dropout_rate = dropout_rate)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_Standardized,Y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] dropout_rate=0.1, learning_rate=0.001 ...........................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  dropout_rate=0.1, learning_rate=0.001, score=1.000, total=   1.0s
[CV] dropout_rate=0.1, learning_rate=0.001 ...........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


[CV]  dropout_rate=0.1, learning_rate=0.001, score=0.750, total=   1.1s
[CV] dropout_rate=0.1, learning_rate=0.001 ...........................


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    7.2s remaining:    0.0s


[CV]  dropout_rate=0.1, learning_rate=0.01, score=0.796, total=   0.8s
[CV] dropout_rate=0.1, learning_rate=0.01 ............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    8.1s remaining:    0.0s


[CV]  dropout_rate=0.1, learning_rate=0.01, score=0.757, total=   1.3s
[CV] dropout_rate=0.1, learning_rate=0.1 .............................
[CV] . dropout_rate=0.1, learning_rate=0.1, score=1.000, total=   0.8s
[CV] dropout_rate=0.1, learning_rate=0.1 .............................
[CV] . dropout_rate=0.1, learning_rate=0.1, score=0.750, total=   0.8s
[CV] dropout_rate=0.1, learning_rate=0.1 .............................
[CV] . dropout_rate=0.1, learning_rate=0.1, score=0.534, total=   0.8s
[CV] dropout_rate=0.1, learning_rate=0.1 .............................
[CV] . dropout_rate=0.1, learning_rate=0.1, score=0.806, total=   0.8s
[CV] dropout_rate=0.1, learning_rate=0.1 .............................
[CV] . dropout_rate=0.1, learning_rate=0.1, score=0.825, total=   0.8s
[CV] dropout_rate=0.2, learning_rate=0.001 ...........................
[CV]  dropout_rate=0.2, learning_rate=0.001, score=1.000, total=   0.9s
[CV] dropout_rate=0.2, learning_rate=0.001 ...........................
[CV] 

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   39.5s finished


In [65]:
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param)) 

Best : 0.8082337617874146, using {'dropout_rate': 0.1, 'learning_rate': 0.01}
0.7305825233459473,0.15435061319000673 with: {'dropout_rate': 0.1, 'learning_rate': 0.001}
0.8082337617874146,0.09365435155842775 with: {'dropout_rate': 0.1, 'learning_rate': 0.01}
0.7830097079277039,0.14999606258875525 with: {'dropout_rate': 0.1, 'learning_rate': 0.1}
0.7325242757797241,0.15375710605921813 with: {'dropout_rate': 0.2, 'learning_rate': 0.001}
0.7635922312736512,0.12375880163905861 with: {'dropout_rate': 0.2, 'learning_rate': 0.01}
0.7596900582313537,0.12469210846217552 with: {'dropout_rate': 0.2, 'learning_rate': 0.1}
0.7305825233459473,0.15435061319000673 with: {'dropout_rate': 0.3, 'learning_rate': 0.001}
0.7305825233459473,0.15435061319000673 with: {'dropout_rate': 0.3, 'learning_rate': 0.01}
0.7383495211601258,0.1525569895928302 with: {'dropout_rate': 0.3, 'learning_rate': 0.1}


Tuning of Hyperparameters:- Activation Function and Kernel Initializer

In [66]:
def create_model(activation_function,init):
    model = Sequential()
    model.add(Dense(8,input_dim = 28,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(4,input_dim = 28,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 10)

# Define the grid search parameters
activation_function = ['softmax','relu','tanh','linear']
init = ['uniform','normal','zero']

# Make a dictionary of the grid search parameters
param_grids = dict(activation_function = activation_function,init = init)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_Standardized,Y)



Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] activation_function=softmax, init=uniform .......................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  activation_function=softmax, init=uniform, score=1.000, total=   0.8s
[CV] activation_function=softmax, init=uniform .......................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV]  activation_function=softmax, init=uniform, score=0.750, total=   0.8s
[CV] activation_function=softmax, init=uniform .......................


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    7.0s remaining:    0.0s


[CV]  activation_function=softmax, init=normal, score=0.320, total=   0.9s
[CV] activation_function=softmax, init=normal ........................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    7.8s remaining:    0.0s


[CV]  activation_function=softmax, init=normal, score=0.699, total=   0.8s
[CV] activation_function=softmax, init=zero ..........................
[CV]  activation_function=softmax, init=zero, score=1.000, total=   0.8s
[CV] activation_function=softmax, init=zero ..........................
[CV]  activation_function=softmax, init=zero, score=0.750, total=   0.8s
[CV] activation_function=softmax, init=zero ..........................
[CV]  activation_function=softmax, init=zero, score=0.524, total=   0.8s
[CV] activation_function=softmax, init=zero ..........................
[CV]  activation_function=softmax, init=zero, score=0.680, total=   0.8s
[CV] activation_function=softmax, init=zero ..........................
[CV]  activation_function=softmax, init=zero, score=0.699, total=   0.8s
[CV] activation_function=relu, init=uniform ..........................
[CV]  activation_function=relu, init=uniform, score=1.000, total=   0.8s
[CV] activation_function=relu, init=uniform .................

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   51.2s finished


In [67]:
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
 print('{},{} with: {}'.format(mean, stdev, param)) 

Best : 0.7403099298477173, using {'activation_function': 'tanh', 'init': 'normal'}
0.7305825233459473,0.15435061319000673 with: {'activation_function': 'softmax', 'init': 'uniform'}
0.6587378680706024,0.22758449693740784 with: {'activation_function': 'softmax', 'init': 'normal'}
0.7305825233459473,0.15435061319000673 with: {'activation_function': 'softmax', 'init': 'zero'}
0.7305825233459473,0.15435061319000673 with: {'activation_function': 'relu', 'init': 'uniform'}
0.7305825233459473,0.15435061319000673 with: {'activation_function': 'relu', 'init': 'normal'}
0.7305825233459473,0.15435061319000673 with: {'activation_function': 'relu', 'init': 'zero'}
0.7364451050758362,0.13511672048688111 with: {'activation_function': 'tanh', 'init': 'uniform'}
0.7403099298477173,0.13810627023335728 with: {'activation_function': 'tanh', 'init': 'normal'}
0.7305825233459473,0.15435061319000673 with: {'activation_function': 'tanh', 'init': 'zero'}
0.7345967054367065,0.1122770952768655 with: {'activation

Tuning of Hyperparameter :-Number of Neurons in activation layer

In [68]:
def create_model(neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 28,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 10)

# Define the grid search parameters

neuron1 = [4,8,16]
neuron2 = [2,4,8]

# Make a dictionary of the grid search parameters

param_grids = dict(neuron1 = neuron1,neuron2 = neuron2)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_Standardized,Y)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ neuron1=4, neuron2=2, score=0.952, total=   0.8s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


[CV] ................ neuron1=4, neuron2=2, score=0.750, total=   0.8s
[CV] neuron1=4, neuron2=2 ............................................


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    6.9s remaining:    0.0s


[CV] ................ neuron1=4, neuron2=4, score=0.680, total=   0.8s
[CV] neuron1=4, neuron2=4 ............................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    7.7s remaining:    0.0s


[CV] ................ neuron1=4, neuron2=4, score=0.709, total=   0.8s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=1.000, total=   0.8s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=0.750, total=   0.9s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=0.563, total=   0.8s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=0.680, total=   1.9s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=0.718, total=   1.2s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ................ neuron1=8, neuron2=2, score=0.971, total=   0.9s
[CV] neuron1=8, neuron2=2 ............................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   40.6s finished


In [70]:
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param)) 

Best : 0.7578230023384094, using {'neuron1': 16, 'neuron2': 8}
0.7365011096000671,0.1195067706156286 with: {'neuron1': 4, 'neuron2': 2}
0.7402912497520446,0.14395476155459408 with: {'neuron1': 4, 'neuron2': 4}
0.7422330021858216,0.1435810475080406 with: {'neuron1': 4, 'neuron2': 8}
0.7306572079658509,0.1338748622404889 with: {'neuron1': 8, 'neuron2': 2}
0.7500373363494873,0.12139135458996753 with: {'neuron1': 8, 'neuron2': 4}
0.748114264011383,0.11693889832238416 with: {'neuron1': 8, 'neuron2': 8}
0.7365384578704834,0.10760208522429719 with: {'neuron1': 16, 'neuron2': 2}
0.7404032945632935,0.1103253558215337 with: {'neuron1': 16, 'neuron2': 4}
0.7578230023384094,0.10939615887766888 with: {'neuron1': 16, 'neuron2': 8}



### ***Training model with optimum values of Hyperparameters***



In [71]:
from sklearn.metrics import classification_report, accuracy_score

# Defining the model

def create_model():
    model = Sequential()
    model.add(Dense(16,input_dim = 28,kernel_initializer = 'normal',activation = 'linear'))
    model.add(Dropout(0.1))
    model.add(Dense(8,input_dim = 28,kernel_initializer = 'normal',activation = 'linear'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = 0.01) #sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 20,epochs = 50)

# Fitting the model

model.fit(X_Standardized,Y)

# Predicting using trained model

y_predict = model.predict(X_Standardized)

# Printing the metrics
print(accuracy_score(Y,y_predict))

0.9825918762088974


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
